In [27]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import urllib
import datetime
from sqlalchemy import create_engine
from functools import reduce
   

In [3]:
path = r'C:\Users\phol\Desktop\test\Lily\Lilycongviec\assignment\3shape'
os.chdir(path)
all_files = glob.glob(path + "/*.csv")

df_list = []
for filename in all_files:
    #Read files
    df = pd.read_csv(filename,index_col=None,sep="#")
    # Remove duplicates if any
    df.drop_duplicates()
    shortFileName = filename.split('\cp_prod_ev_')[1] 
    for eachName in shortFileName:
        shortFileName = shortFileName.split('_202010281322.csv')[0] 
    print(shortFileName, df.shape)
    df_list.append(df)

form_submitted (18752, 15)
link_clicked (1154365, 15)
page_scrolled (198412, 15)
page_viewed (2058871, 17)
pdf_viewed (83095, 15)
site_visited (1380057, 22)
video_watched (91378, 15)


In [4]:
form_submitted = df_list[0]
form_submitted.columns

Index(['country', 'session_count', 'campaign', 'date', 'medium', 'page_path',
       'source', 'form_submitted', 'form_submitted_unique', 'subdomain',
       'full_url', 'form_name', 'language_version', 'channel_grouping',
       'status'],
      dtype='object')

In [8]:
link_clicked = df_list[1]
link_clicked.head(5)

,country,session_count,campaign,date,medium,page_path,source,link_clicked,event_label,link_clicked_unique,subdomain,full_url,language_version,channel_grouping,status
0,United States,2,(not set),2020-02-29 00:00:00,organic,/knowledge-center/news-and-press/news/2015/tri...,google,1,https://ww2.3shape.com/pt-br/knowledge-center/...,1,ww2.3shape.com,ww2.3shape.com/knowledge-center/news-and-press...,pt,Organic Search,Live
1,United States,2,(not set),2020-03-01 00:00:00,organic,/,duckduckgo,1,(not set),0,www.3shape.com,www.3shape.com/,en,Organic Search,Live
2,United States,2,(not set),2020-03-01 00:00:00,organic,/news/2014/3-step-guide-to-making-model-free-c...,google,3,(not set),1,www.3shape.com,www.3shape.com/news/2014/3-step-guide-to-makin...,en,Organic Search,Live
3,United States,2,(not set),2020-03-01 00:00:00,organic,/scanners/lab,google,1,(not set),1,www.3shape.com,www.3shape.com/scanners/lab,en,Organic Search,Live
4,United States,2,(not set),2020-03-01 00:00:00,organic,/scanners/trios,google,2,(not set),1,www.3shape.com,www.3shape.com/scanners/trios,en,Organic Search,Live


In [22]:
# 1. Subset data with live status only
# 2. Group by key dimensions, sum aggregation metrics

form_submitted = df_list[0].loc[df_list[0].status=='Live',:]\
                            .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'form_submitted':'sum'})

link_clicked = df_list[1].loc[df_list[1].status=='Live',:]\
                            .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'link_clicked':'sum'})

page_scrolled = df_list[2].loc[df_list[2].status=='Live',:]\
                            .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'page_scrolled':'sum'})

page_viewed = df_list[3].loc[df_list[3].status=='Live',:]\
                        .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'page_viewed':'sum'})

pdf_viewed = df_list[4].loc[df_list[4].status=='Live',:]\
                        .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'pdf_viewed':'sum'})

site_visited = df_list[5].loc[df_list[5].status=='Live',:]\
                        .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'site_visited':'sum'})

video_watched = df_list[6].loc[df_list[6].status=='Live',:]\
                        .groupby(['country', 'campaign','date','channel_grouping'])\
                            .agg({'video_watched':'sum'})

joinList = [form_submitted,link_clicked,page_scrolled, page_viewed, pdf_viewed, site_visited, video_watched]


allDF = reduce(lambda left, right: pd.merge(left, right,
                                   on=['country','campaign','date','channel_grouping'],
                                   how='outer'),joinList)
allDF.reset_index(inplace=True)  
allDF = allDF.fillna(0).sort_values(by='date')
allDF[allDF.columns[4:]] = allDF[allDF.columns[4:]].astype(int)
allDF.head(50)

,country,campaign,date,channel_grouping,form_submitted,link_clicked,page_scrolled,page_viewed,pdf_viewed,site_visited,video_watched
89146,Romania,(not set),2019-01-01 00:00:00,Organic Search,0,3,0,7,3,4,0
166543,India,Generic|IN|Lab|Scanners,2019-01-01 00:00:00,Paid Search,0,0,0,1,0,1,0
21079,Bahrain,(not set),2019-01-01 00:00:00,Organic Search,0,3,0,4,0,3,0
152782,Croatia,(not set),2019-01-01 00:00:00,Organic Search,0,0,0,1,0,1,0
21220,Bangladesh,(not set),2019-01-01 00:00:00,Organic Search,0,2,0,3,0,1,0
52850,Greece,(not set),2019-01-01 00:00:00,Organic Search,0,13,0,7,1,4,0
150694,Chile,(not set),2019-01-01 00:00:00,Direct,0,0,0,2,0,1,0
34945,Colombia,(not set),2019-01-01 00:00:00,Organic Search,0,6,0,8,0,5,0
139133,Andorra,(not set),2019-01-01 00:00:00,Organic Search,0,0,0,1,0,1,0
110510,Turkey,(not set),2019-01-01 00:00:00,Organic Search,0,3,0,13,1,10,1


In [23]:
fileoutput = path+'/media.csv'

if os.path.exists(fileoutput):
    os.remove(fileoutput)
    allDF.to_csv(path+'/media.csv', index=False)
    print('done')
else:
    allDF.to_csv(path+'/media.csv', index=False)
    print('done')

done
